# Purpose
from https://nyheder.tv2.dk/kommunalvalg/find-en-kandidat
to a .csv file
target municipalities
- København
- Frederiksberg
- Hvidovre
- Tårnby
- Dragør
- Rødovre
- Brøndby
- Gentofte
- Gladsaxe
- Herlev

# requirements
- beautiful soup 4
- pandas
- lxml

In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from enum import Enum

URL = "https://nyheder.tv2.dk/kommunalvalg/find-en-kandidat"
municipalities = ["København", "Frederiksberg", "Hvidovre", "Tårnby", "Dragør", "Rødovre", "Brøndby", "Gentofte", "Gladsaxe", "Herlev"]
topics = ["Beredskab og sikkerhed", #order may differ
        "Erhverv / administration", 
        "x Kommune", # todo: insert kommune
        "Kultur, idræt og fritid",
        "Miljø og klima",
        "Skat",
        "Skole / dagtilbud for børn",
        "Social- og integrationsområdet",
        "Sundhed",
        "Trafik",
        "Ældre"]

class scale(Enum):
    "Helt uenig" = 0
    "Overvejende uenig" = 1
    "Hverken/eller" = 2
    "Overvejende enig" = 3
    "Helt enig" = 4

In [ ]:
def get_candidate_info()
    # handle case with no info given
    # name
    # num votes
    # candidate ID
    # elected bool
    # age
    # party
    # relationship status
    # num kids
    # area
    # job
    # test and answers
    # short text
    # longer text

    # return string with info

In [ ]:
for municipality in municipalities:
    # for party
        # call get_candidate_info